# Database Dump Transformation and Cleaning

This notebook transforms the PostgreSQL dump file into cleaned pandas DataFrames for analysis.

## Setup and Dependencies

In [1]:
# # Install required dependencies
# import sys
# import subprocess

# def install_package(package):
#     """Install a package using pip."""
#     try:
#         __import__(package)
#         print(f"✓ {package} is already installed")
#     except ImportError:
#         print(f"Installing {package}...")
#         subprocess.check_call([sys.executable, "-m", "pip", "install", package, "--quiet"])
#         print(f"✓ {package} installed successfully")

# # Install required packages
# required_packages = [
#     "pandas",
#     "numpy",
#     "jupyter",
#     "ipython"
# ]

# print("Checking and installing dependencies...\n")
# for package in required_packages:
#     install_package(package)

# print("\n✓ All dependencies are ready!")
# print("\nNote: If you need to convert PostgreSQL custom format dumps, you'll also need:")
# print("  - PostgreSQL client tools (pg_restore)")
# print("  - macOS: brew install postgresql")
# print("  - Ubuntu: sudo apt-get install postgresql-client")

In [2]:
import pandas as pd
import numpy as np
import json
import re
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

Pandas version: 3.0.0
NumPy version: 2.4.1


## Step 1: Locate and Verify Dump File

In [3]:
# Find the dump file
dump_files = [
    Path.home() / "Downloads" / "b078-20260113-215725.dump",
    Path("/workspace") / "heroku_psql_181025.dump",
    Path("./heroku_psql_181025.dump"),
    Path("./b078-20260113-215725.dump")
]

dump_file = None
for df in dump_files:
    if df.exists():
        dump_file = df
        break

if dump_file:
    print(f"Found dump file: {dump_file}")
    print(f"File size: {dump_file.stat().st_size / 1024:.2f} KB")
    
    # Check file format
    with open(dump_file, 'rb') as f:
        header = f.read(5)
        if header == b'PGDMP':
            print("Format: PostgreSQL custom format (requires pg_restore)")
            file_format = 'custom'
        else:
            print("Format: Plain SQL")
            file_format = 'sql'
else:
    print("ERROR: Could not find dump file.")
    print("Please ensure the dump file is in one of these locations:")
    for df in dump_files:
        print(f"  - {df}")

Found dump file: /Users/johndriscoll/Downloads/b078-20260113-215725.dump
File size: 248.83 KB
Format: PostgreSQL custom format (requires pg_restore)


## Manual Conversion Instructions

If automatic conversion failed, convert the dump file manually using one of these methods:

### Method 1: Using PostgreSQL 16 (Recommended)

Since PostgreSQL 16 is "keg-only" (not in PATH), use the full path:

**For Apple Silicon Macs:**
```bash
/opt/homebrew/opt/postgresql@17/bin/pg_restore --no-owner --no-privileges -f ~/Downloads/b078-20260113-215725.sql ~/Downloads/b078-20260113-215725.dump
```

**For Intel Macs:**
```bash
/usr/local/opt/postgresql@16/bin/pg_restore --no-owner --no-privileges -f ~/Downloads/b078-20260113-215725.sql ~/Downloads/b078-20260113-215725.dump
```


## Step 2: Convert Dump to SQL (if needed)

This cell will automatically convert the PostgreSQL custom format dump to SQL format.

In [4]:
import subprocess
import os

# Check if we need to convert
if 'dump_file' in globals() and 'file_format' in globals() and file_format == 'custom':
    print("="*60)
    print("CONVERTING CUSTOM FORMAT DUMP TO SQL")
    print("="*60)
    
    # Define output SQL file
    sql_file = dump_file.with_suffix('.sql')
    
    # Check if SQL file already exists
    if sql_file.exists():
        print(f"\n✓ SQL file already exists: {sql_file}")
        print(f"  File size: {sql_file.stat().st_size / 1024:.2f} KB")
        print("  Using existing file. Skipping conversion.")
        dump_file = sql_file
        file_format = 'sql'
    else:
        print(f"\nInput dump file: {dump_file}")
        print(f"Output SQL file: {sql_file}")
        
        # Try to find pg_restore in various locations
        pg_restore_paths = [
            # PostgreSQL 17 (newest)
            '/opt/homebrew/opt/postgresql@17/bin/pg_restore',
            '/usr/local/opt/postgresql@17/bin/pg_restore',
            # PostgreSQL 16
            '/opt/homebrew/opt/postgresql@16/bin/pg_restore',
            '/usr/local/opt/postgresql@16/bin/pg_restore',
            # PostgreSQL 15
            '/opt/homebrew/opt/postgresql@15/bin/pg_restore',
            '/usr/local/opt/postgresql@15/bin/pg_restore',
            # Standard locations
            '/opt/homebrew/bin/pg_restore',
            '/usr/local/bin/pg_restore',
            'pg_restore',  # In PATH
        ]
        
        pg_restore_path = None
        version_info = None
        
        print("\nSearching for pg_restore...")
        for path in pg_restore_paths:
            try:
                if path == 'pg_restore':
                    # Check if it's in PATH
                    result = subprocess.run(
                        ['which', 'pg_restore'],
                        capture_output=True,
                        text=True
                    )
                    if result.returncode == 0:
                        path = result.stdout.strip()
                    else:
                        continue
                
                if os.path.exists(path):
                    # Check version
                    result = subprocess.run(
                        [path, '--version'],
                        capture_output=True,
                        text=True,
                        check=True
                    )
                    pg_restore_path = path
                    version_info = result.stdout.strip()
                    print(f"✓ Found: {version_info}")
                    print(f"  Location: {path}")
                    break
            except Exception as e:
                continue
        
        if pg_restore_path:
            print(f"\nConverting dump to SQL...")
            print(f"  Command: {pg_restore_path} --no-owner --no-privileges -f {sql_file} {dump_file}")
            
            try:
                result = subprocess.run(
                    [pg_restore_path, '--no-owner', '--no-privileges', '-f', str(sql_file), str(dump_file)],
                    capture_output=True,
                    text=True,
                    timeout=300  # 5 minute timeout
                )
                
                if result.returncode == 0:
                    if sql_file.exists():
                        print(f"\n✓✓✓ CONVERSION SUCCESSFUL! ✓✓✓")
                        print(f"  Created: {sql_file}")
                        print(f"  File size: {sql_file.stat().st_size / 1024:.2f} KB")
                        dump_file = sql_file
                        file_format = 'sql'
                        print("\n✓ Ready to proceed with parsing!")
                    else:
                        print(f"\n✗ Conversion reported success but SQL file not found!")
                        print(f"  Expected location: {sql_file}")
                else:
                    error_msg = result.stderr.strip() if result.stderr else result.stdout.strip()
                    print(f"\n✗ CONVERSION FAILED")
                    print(f"  Error: {error_msg}")
                    
                    if 'unsupported version' in error_msg.lower():
                        print("\n" + "="*60)
                        print("VERSION INCOMPATIBILITY")
                        print("="*60)
                        print(f"\nThe dump file format version is not supported by {version_info}.")
                        print("\nSOLUTIONS:")
                        print("\n1. Try a newer PostgreSQL version (17 or 18)")
                        print("   brew install postgresql@17")
                        print(f"   /opt/homebrew/opt/postgresql@17/bin/pg_restore -f {sql_file} {dump_file}")
                        print("\n2. Use Docker:")
                        print(f"   docker run --rm -v ~/Downloads:/data postgres:17 \\")
                        print(f"     pg_restore -f /data/{sql_file.name} /data/{dump_file.name}")
                        print("\n3. Check if SQL file was already created manually")
                    else:
                        print("\nPlease check the error message above and try manual conversion.")
                        print("See the manual conversion instructions in the previous cell.")
            except subprocess.TimeoutExpired:
                print("\n✗ Conversion timed out after 5 minutes.")
                print("  The dump file might be very large or there's an issue.")
            except Exception as e:
                print(f"\n✗ Error during conversion: {e}")
        else:
            print("\n✗ pg_restore not found!")
            print("\nPlease install PostgreSQL client tools:")
            print("  brew install postgresql@17")
            print("\nOr use Docker (see manual conversion instructions above).")
            print("\nAfter manual conversion, set:")
            print(f"  dump_file = Path('{sql_file}')")
            print(f"  file_format = 'sql'")
            print("\nThen proceed to the next cell.")

elif 'dump_file' in globals() and 'file_format' in globals() and file_format == 'sql':
    print("✓ Dump file is already in SQL format.")
    print(f"  File: {dump_file}")
    if isinstance(dump_file, Path) and dump_file.exists():
        print(f"  File size: {dump_file.stat().st_size / 1024:.2f} KB")
    print("\nReady to proceed with parsing!")
else:
    print("⚠ Cannot convert: dump_file or file_format not set.")
    print("Please run the previous cell to locate and detect the dump file format.")

CONVERTING CUSTOM FORMAT DUMP TO SQL

✓ SQL file already exists: /Users/johndriscoll/Downloads/b078-20260113-215725.sql
  File size: 629.43 KB
  Using existing file. Skipping conversion.


In [5]:
! /opt/homebrew/opt/postgresql@17/bin/pg_restore --no-owner --no-privileges -v -f ~/Downloads/b078-20260113-215725.sql ~/Downloads/b078-20260113-215725.dump

pg_restore: creating SCHEMA "public"
pg_restore: creating EXTENSION "pg_stat_statements"
pg_restore: creating COMMENT "EXTENSION "pg_stat_statements""
pg_restore: creating TABLE "public.alembic_version"
pg_restore: creating TABLE "public.assignment_session_activity"
pg_restore: creating TABLE "public.attention_check_question"
pg_restore: creating TABLE "public.attention_check_response"
pg_restore: creating TABLE "public.attention_check_scenarios"
pg_restore: creating TABLE "public.auth"
pg_restore: creating TABLE "public.channel"
pg_restore: creating TABLE "public.channel_member"
pg_restore: creating TABLE "public.chat"
pg_restore: creating TABLE "public.chatidtag"
pg_restore: creating TABLE "public.child_profile"
pg_restore: creating TABLE "public.config"
pg_restore: creating SEQUENCE "public.config_id_seq"
pg_restore: creating SEQUENCE OWNED BY "public.config_id_seq"
pg_restore: creating TABLE "public.consent_audit"
pg_restore: creating TABLE "public.document"
pg_restore: creating TA

In [6]:
# Set path to SQL file (after conversion)
from pathlib import Path

# Point to the converted SQL file - EXPAND PATH PROPERLY
dump_file = Path("~/Downloads/b078-20260113-215725.sql").expanduser()
file_format = 'sql'

print(f"✓ dump_file set to: {dump_file}")
print(f"✓ file_format set to: {file_format}")
print(f"✓ File exists: {dump_file.exists()}")

if dump_file.exists():
    print(f"✓ File size: {dump_file.stat().st_size / 1024:.2f} KB")
    # Check if it's actually SQL or still binary
    with open(dump_file, 'rb') as f:
        header = f.read(100)
        if header.startswith(b'PGDMP'):
            print("⚠️  WARNING: File is still in PostgreSQL custom format! Run conversion cell first.")
        elif b'COPY' in header or b'CREATE' in header or b'INSERT' in header:
            print("✓ File appears to be SQL format")
        else:
            print("⚠️  File format unknown")
else:
    print("✗ File does not exist! Check the path or run the conversion cell first.")

✓ dump_file set to: /Users/johndriscoll/Downloads/b078-20260113-215725.sql
✓ file_format set to: sql
✓ File exists: True
✓ File size: 629.43 KB
⚠️  File format unknown


## Step 3: Parse SQL Dump and Extract Tables

We'll parse the SQL dump to extract table data into pandas DataFrames.

In [7]:
def parse_sql_dump(sql_file_path):
    """Parse SQL dump file and extract table data.
    
    This function handles PostgreSQL SQL dumps with COPY commands.
    It extracts table names, columns, and data into pandas DataFrames.
    """
    import re
    import json
    import pandas as pd
    from pathlib import Path
    
    sql_file_path = Path(sql_file_path)
    print(f"Reading SQL file: {sql_file_path}")
    print(f"File size: {sql_file_path.stat().st_size / 1024:.2f} KB")
    
    # Read file content
    with open(sql_file_path, 'rb') as f:
        content = f.read()
    
    # Check if it's actually SQL
    if content.startswith(b'PGDMP'):
        raise ValueError("File is in PostgreSQL custom format! Please run pg_restore conversion first.")
    
    # Try to decode with different encodings
    try:
        text_content = content.decode('utf-8')
    except UnicodeDecodeError:
        print("⚠️  UTF-8 decode failed, trying latin-1...")
        try:
            text_content = content.decode('latin-1', errors='ignore')
        except:
            print("⚠️  latin-1 failed, using utf-8 with errors='replace'")
            text_content = content.decode('utf-8', errors='replace')
    
    print(f"Decoded {len(text_content)} characters")
    
    tables_data = {}
    
    # Find all COPY statements - handle various formats
    # Format: COPY "public"."table_name" (columns) FROM stdin;
    # Or: COPY public.table_name (columns) FROM stdin;
    copy_pattern = r'COPY\s+(?:"public"\.)?(?:")?(\w+)(?:")?\s*\(([^)]+)\)\s+FROM stdin;'
    copy_matches = list(re.finditer(copy_pattern, text_content, re.MULTILINE | re.IGNORECASE))
    
    print(f"\nFound {len(copy_matches)} COPY statements\n")
    
    if len(copy_matches) == 0:
        print("⚠️  No COPY statements found. Checking for INSERT statements...")
        # Try to find INSERT statements as fallback
        insert_pattern = r'INSERT INTO\s+(?:"public"\.)?(?:")?(\w+)(?:")?'
        insert_matches = list(re.finditer(insert_pattern, text_content, re.MULTILINE | re.IGNORECASE))
        print(f"Found {len(insert_matches)} INSERT statements")
        if len(insert_matches) == 0:
            print("⚠️  No data extraction possible from this file")
            return {}
    
    for i, copy_match in enumerate(copy_matches):
        table_name = copy_match.group(1)
        columns_str = copy_match.group(2)
        
        # Parse column names - handle quoted and unquoted
        columns = []
        for col in columns_str.split(','):
            col = col.strip()
            # Remove quotes if present
            col = col.strip('"').strip("'")
            columns.append(col)
        
        print(f"Processing table: {table_name}")
        print(f"  Columns ({len(columns)}): {', '.join(columns[:5])}{'...' if len(columns) > 5 else ''}")
        
        # Find data section - between COPY command and next COPY or end marker
        start_pos = copy_match.end()
        
        # Find end position
        if i + 1 < len(copy_matches):
            # End at next COPY statement
            end_pos = copy_matches[i + 1].start()
        else:
            # Find the \. marker that ends COPY data
            end_marker_matches = list(re.finditer(r'^\\\.$', text_content[start_pos:], re.MULTILINE))
            if end_marker_matches:
                end_pos = start_pos + end_marker_matches[0].start()
            else:
                end_pos = len(text_content)
        
        data_section = text_content[start_pos:end_pos]
        
        # Parse tab-separated values
        rows = []
        lines = data_section.strip().split('\n')
        
        skipped_lines = 0
        for line in lines:
            line = line.strip()
            
            # Skip empty lines and end markers
            if not line or line.startswith('\\'):
                continue
            
            # Skip SQL commands that might be mixed in
            if line.upper().startswith(('CREATE', 'ALTER', 'DROP', 'INSERT', 'UPDATE', 'DELETE', 'SELECT', '--', '/*')):
                continue
            
            # Split by tab - PostgreSQL COPY uses tab as delimiter
            values = line.split('\t')
            
            # Check if we have the right number of columns
            if len(values) != len(columns):
                skipped_lines += 1
                if skipped_lines <= 3:  # Only show first few mismatches
                    print(f"    ⚠️  Line has {len(values)} values but expected {len(columns)}, skipping")
                continue
            
            # Parse values
            row = {}
            for col, val in zip(columns, values):
                # Handle PostgreSQL NULL marker
                if val == '\\N':
                    row[col] = None
                else:
                    # Try to parse as JSON for JSON/JSONB columns
                    if val and (val.startswith('{') or val.startswith('[')):
                        try:
                            row[col] = json.loads(val)
                        except json.JSONDecodeError:
                            # Not valid JSON, keep as string
                            row[col] = val
                    else:
                        row[col] = val
            
            rows.append(row)
        
        if skipped_lines > 3:
            print(f"    ⚠️  Skipped {skipped_lines} total lines with column mismatches")
        
        # Create DataFrame if we have data
        if rows:
            tables_data[table_name] = pd.DataFrame(rows)
            print(f"  ✓ Extracted {len(rows)} rows\n")
        else:
            print(f"  ⚠️  No data rows found\n")
    
    print(f"\n{'='*60}")
    print(f"Successfully extracted {len(tables_data)} tables:")
    for table_name, df in tables_data.items():
        print(f"  • {table_name}: {len(df)} rows × {len(df.columns)} columns")
    print(f"{'='*60}\n")
    
    return tables_data

# Actually parse the dump and create raw_dataframes
if 'dump_file' not in globals() or 'file_format' not in globals():
    print("⚠️  ERROR: dump_file and file_format not set!")
    print("Please run Cell 9 first to set these variables.")
else:
    if not dump_file.exists():
        print(f"⚠️  ERROR: SQL file not found at: {dump_file}")
        print("Please check the path or run the conversion cell first.")
    else:
        print("Parsing SQL dump...")
        raw_dataframes = parse_sql_dump(dump_file)
        print(f"\n✓ raw_dataframes created with {len(raw_dataframes)} tables")

Parsing SQL dump...
Reading SQL file: /Users/johndriscoll/Downloads/b078-20260113-215725.sql
File size: 629.43 KB
Decoded 635989 characters

Found 36 COPY statements

Processing table: alembic_version
  Columns (1): version_num
  ✓ Extracted 1 rows

Processing table: assignment_session_activity
  Columns (6): id, user_id, session_number, active_ms_delta, cumulative_ms...
  ✓ Extracted 521 rows

Processing table: attention_check_question
  Columns (5): id, prompt, options, correct_option, created_at
  ⚠️  No data rows found

Processing table: attention_check_response
  Columns (7): id, user_id, session_number, question_id, response...
  ⚠️  No data rows found

Processing table: attention_check_scenarios
  Columns (13): scenario_id, prompt_text, response_text, trait_theme, trait_phrase...
  ✓ Extracted 50 rows

Processing table: auth
  Columns (4): id, email, password, active
  ✓ Extracted 4 rows

Processing table: channel
  Columns (10): id, user_id, name, description, data...
  ⚠️  No 

## Step 4: Define Relevant Tables

We'll focus on these tables for analysis:

In [8]:
RELEVANT_TABLES = [
    'user',
    'chat',
    'message',
    'child_profile',
    'selection',
    'moderation_scenario',
    'moderation_session',
    'moderation_applied',
    'moderation_question_answer',
    'exit_quiz_response',
    'scenario_assignments',
    'scenarios',
    'attention_check_scenarios',
    'assignment_session_activity',
]

print("Relevant tables for analysis:")
for table in RELEVANT_TABLES:
    status = "✓" if table in raw_dataframes else "✗"
    count = len(raw_dataframes[table]) if table in raw_dataframes else 0
    print(f"  {status} {table}: {count} rows")

Relevant tables for analysis:
  ✓ user: 4 rows
  ✗ chat: 0 rows
  ✗ message: 0 rows
  ✓ child_profile: 4 rows
  ✓ selection: 47 rows
  ✗ moderation_scenario: 0 rows
  ✓ moderation_session: 32 rows
  ✗ moderation_applied: 0 rows
  ✗ moderation_question_answer: 0 rows
  ✓ exit_quiz_response: 5 rows
  ✓ scenario_assignments: 55 rows
  ✓ scenarios: 50 rows
  ✓ attention_check_scenarios: 50 rows
  ✓ assignment_session_activity: 521 rows


## Step 5: Clean and Transform Data

Now we'll clean and transform each relevant table systematically.

### 5.1: Helper Functions for Data Cleaning

In [9]:
def convert_timestamps(df, timestamp_cols=None):
    """Convert timestamp columns to datetime, handling different scales.
    
    This function handles:
    - Nanosecond timestamps (> 1e15)
    - Millisecond timestamps (> 1e12)
    - Second timestamps (> 1e9)
    - ISO format strings
    - Already datetime objects
    """
    import pandas as pd
    import numpy as np
    from datetime import datetime
    
    df = df.copy()
    
    if timestamp_cols is None:
        # Auto-detect timestamp columns
        timestamp_cols = [col for col in df.columns 
                         if 'at' in col.lower() or 'time' in col.lower() or 'date' in col.lower()]
    
    print(f"Converting timestamps in columns: {timestamp_cols}")
    
    for col in timestamp_cols:
        if col not in df.columns:
            print(f"  ⚠️  Column {col} not found, skipping")
            continue
        
        # Skip if already datetime
        if pd.api.types.is_datetime64_any_dtype(df[col]):
            print(f"  ✓ {col} already datetime")
            continue
            
        try:
            # Convert to numeric first, handling NaN
            numeric_vals = pd.to_numeric(df[col], errors='coerce')
            
            # Check if all NaN
            if numeric_vals.isna().all():
                print(f"  ⚠️  {col}: All values are NaN or non-numeric")
                df[col + '_datetime'] = pd.NaT
                continue
            
            # Get a sample non-NaN value to determine scale
            sample_val = numeric_vals.dropna().iloc[0] if len(numeric_vals.dropna()) > 0 else None
            
            if sample_val is None:
                print(f"  ⚠️  {col}: No valid numeric values")
                df[col + '_datetime'] = pd.NaT
                continue
            
            # Determine the scale based on magnitude
            # Nanosecond timestamps are typically > 1e15
            # Millisecond timestamps are typically > 1e12 but < 1e15
            # Second timestamps are typically > 1e9 but < 1e12
            
            if sample_val > 1e15:
                # Nanoseconds
                print(f"  • {col}: Detected nanosecond timestamps (sample: {sample_val:.0f})")
                df[col + '_datetime'] = pd.to_datetime(numeric_vals, unit='ns', errors='coerce')
            elif sample_val > 1e12:
                # Milliseconds
                print(f"  • {col}: Detected millisecond timestamps (sample: {sample_val:.0f})")
                df[col + '_datetime'] = pd.to_datetime(numeric_vals, unit='ms', errors='coerce')
            elif sample_val > 1e9:
                # Seconds
                print(f"  • {col}: Detected second timestamps (sample: {sample_val:.0f})")
                df[col + '_datetime'] = pd.to_datetime(numeric_vals, unit='s', errors='coerce')
            else:
                # Try direct datetime conversion (might be string format)
                print(f"  • {col}: Attempting direct datetime conversion (sample: {sample_val})")
                df[col + '_datetime'] = pd.to_datetime(df[col], errors='coerce')
            
            # Report conversion results
            converted = df[col + '_datetime'].notna().sum()
            total = len(df)
            print(f"    ✓ Converted {converted}/{total} values ({converted/total*100:.1f}%)")
            
            # Show sample of converted values
            if converted > 0:
                sample_dt = df[col + '_datetime'].dropna().iloc[0]
                print(f"    Sample: {sample_dt}")
            
        except Exception as e:
            print(f"  ✗ Error converting {col}: {e}")
            df[col + '_datetime'] = pd.NaT
    
    return df

print("✓ convert_timestamps function defined")

✓ convert_timestamps function defined


### 5.2: Clean User Table

In [10]:
# Enhanced helper functions with fixes for timestamp and JSON parsing

def clean_strings(df):
    """Clean string columns by removing null bytes and normalizing null values.
    
    Args:
        df: DataFrame to clean
        
    Returns:
        Cleaned DataFrame
    """
    import pandas as pd
    
    df = df.copy()
    
    # Get string columns
    string_cols = df.select_dtypes(include=['object']).columns
    
    for col in string_cols:
        # Remove null bytes
        df[col] = df[col].apply(lambda x: x.replace('\x00', '') if isinstance(x, str) else x)
        
        # Replace literal null strings with None
        df[col] = df[col].replace({
            'null': None,
            'NULL': None,
            'None': None,
            'nan': None,
            'NaN': None,
            '': None
        })
    
    return df


def parse_boolean_column(df, col_name):
    """Convert column to boolean, handling various formats.
    
    Args:
        df: DataFrame to process
        col_name: Name of column to convert
        
    Returns:
        DataFrame with converted column
    """
    import pandas as pd
    
    if col_name not in df.columns:
        print(f"  ⚠️  Column '{col_name}' not found")
        return df
    
    # Map various boolean representations
    bool_map = {
        't': True, 'f': False,
        'true': True, 'false': False,
        'True': True, 'False': False,
        'TRUE': True, 'FALSE': False,
        '1': True, '0': False,
        1: True, 0: False,
        'yes': True, 'no': False,
        'Yes': True, 'No': False,
        'YES': True, 'NO': False
    }
    
    df[col_name] = df[col_name].map(bool_map)
    converted = df[col_name].notna().sum()
    print(f"  • Converted {col_name}: {converted}/{len(df)} values to boolean")
    
    return df


def parse_numeric_column(df, col_name, dtype='float'):
    """Convert column to numeric type.
    
    Args:
        df: DataFrame to process
        col_name: Name of column to convert
        dtype: Target dtype ('int' or 'float')
        
    Returns:
        DataFrame with converted column
    """
    import pandas as pd
    
    if col_name not in df.columns:
        print(f"  ⚠️  Column '{col_name}' not found")
        return df
    
    if dtype == 'int':
        df[col_name] = pd.to_numeric(df[col_name], errors='coerce').astype('Int64')
    else:
        df[col_name] = pd.to_numeric(df[col_name], errors='coerce')
    
    converted = df[col_name].notna().sum()
    print(f"  • Converted {col_name}: {converted}/{len(df)} values to {dtype}")
    
    return df


# Enhanced helper functions with fixes for timestamp and JSON parsing

def parse_json_column(df, col_name, new_col_name=None):
    """Parse JSON column into a new column.
    
    Args:
        df: DataFrame to process
        col_name: Name of column containing JSON strings
        new_col_name: Name for new parsed column (default: {col_name}_parsed)
    
    Returns:
        DataFrame with new parsed column added
    """
    import pandas as pd
    import json
    
    if new_col_name is None:
        new_col_name = f'{col_name}_parsed'
    
    if col_name not in df.columns:
        print(f"  ⚠️  Column '{col_name}' not found")
        return df
    
    def parse_json(val):
        # Already a dict or list
        if isinstance(val, (dict, list)):
            return val
        
        # None or NaN
        if val is None or pd.isna(val):
            return None
        
        # String that needs parsing
        if isinstance(val, str):
            # Handle literal null strings
            if val.lower() in ('null', 'none', 'nan') or val.strip() == '':
                return None
            
            val_stripped = val.strip()
            
            # Try to parse JSON
            if val_stripped.startswith('{') or val_stripped.startswith('['):
                try:
                    return json.loads(val_stripped)
                except json.JSONDecodeError:
                    # Try fixing single quotes
                    try:
                        val_fixed = val_stripped.replace("'", '"')
                        return json.loads(val_fixed)
                    except:
                        return None
        
        return None
    
    df[new_col_name] = df[col_name].apply(parse_json)
    parsed_count = df[new_col_name].notna().sum()
    print(f"  • Parsed {col_name}: {parsed_count}/{len(df)} values")
    
    return df


# Enhanced helper functions with fixes for timestamp and JSON parsing

def convert_timestamps_enhanced(df, timestamp_cols=None):
    """Convert timestamp columns to datetime - enhanced version."""
    df = df.copy()
    
    if timestamp_cols is None:
        timestamp_cols = [col for col in df.columns 
                         if 'at' in col.lower() or 'time' in col.lower()]
    
    for col in timestamp_cols:
        if col not in df.columns:
            continue
            
        try:
            # Convert to numeric first (handles string timestamps)
            numeric_col = pd.to_numeric(df[col], errors='coerce')
            sample_val = numeric_col.dropna()
            
            if len(sample_val) > 0:
                val = sample_val.iloc[0]
                # Determine unit based on magnitude
                if val > 1e15:
                    # Nanoseconds (very large numbers like 1768343088867177168)
                    df[f'{col}_datetime'] = pd.to_datetime(numeric_col / 1e9, unit='s', errors='coerce')
                elif val > 1e12:
                    # Microseconds
                    df[f'{col}_datetime'] = pd.to_datetime(numeric_col / 1e6, unit='s', errors='coerce')
                elif val > 1e9:
                    # Milliseconds
                    df[f'{col}_datetime'] = pd.to_datetime(numeric_col, unit='ms', errors='coerce')
                else:
                    # Seconds
                    df[f'{col}_datetime'] = pd.to_datetime(numeric_col, unit='s', errors='coerce')
            else:
                df[f'{col}_datetime'] = pd.to_datetime(df[col], errors='coerce')
        except Exception as e:
            print(f"    Warning: Could not convert {col}: {e}")
    
    return df

def parse_json_column_enhanced(df, col_name, new_col_name=None):
    """Parse JSON column - enhanced version that handles 'null' strings."""
    if new_col_name is None:
        new_col_name = f'{col_name}_parsed'
    
    if col_name not in df.columns:
        return df
    
    def parse_json(val):
        if isinstance(val, (dict, list)):
            return val
        if val is None or pd.isna(val):
            return None
        if isinstance(val, str):
            if val.lower() == 'null' or val.strip() == '':
                return None
            val_stripped = val.strip()
            if val_stripped.startswith('{') or val_stripped.startswith('['):
                try:
                    return json.loads(val_stripped)
                except:
                    try:
                        val_fixed = val_stripped.replace("'", '"')
                        return json.loads(val_fixed)
                    except:
                        return None
        return None
    
    df[new_col_name] = df[col_name].apply(parse_json)
    return df

# Override the original functions
convert_timestamps = convert_timestamps_enhanced
parse_json_column = parse_json_column_enhanced

print("Enhanced helper functions loaded.")

Enhanced helper functions loaded.


In [11]:
if 'user' in raw_dataframes:
    print("Cleaning user table...")
    df_users = raw_dataframes['user'].copy()
    
    # Basic cleaning
    df_users = clean_strings(df_users)
    df_users = convert_timestamps(df_users)
    
    # Parse JSON fields
    df_users = parse_json_column(df_users, 'info')
    df_users = parse_json_column(df_users, 'settings')
    
    # Convert date_of_birth if present
    if 'date_of_birth' in df_users.columns:
        df_users['date_of_birth'] = pd.to_datetime(df_users['date_of_birth'], errors='coerce')
    
    print(f"  ✓ Cleaned user table: {len(df_users)} rows, {len(df_users.columns)} columns")
    print(f"  Columns: {', '.join(df_users.columns[:10])}..." if len(df_users.columns) > 10 else f"  Columns: {', '.join(df_users.columns)}")
else:
    print("✗ User table not found")
    df_users = pd.DataFrame()

Cleaning user table...
  ✓ Cleaned user table: 4 rows, 27 columns
  Columns: id, name, email, role, profile_image_url, last_active_at, updated_at, created_at, api_key, settings...


In [44]:
df_users.columns

Index(['id', 'name', 'email', 'role', 'profile_image_url', 'last_active_at',
       'updated_at', 'created_at', 'api_key', 'settings', 'info', 'oauth_sub',
       'username', 'bio', 'gender', 'date_of_birth', 'prolific_pid',
       'study_id', 'current_session_id', 'session_number', 'consent_given',
       'last_active_at_datetime', 'updated_at_datetime', 'created_at_datetime',
       'date_of_birth_datetime', 'info_parsed', 'settings_parsed',
       'last_active_at_datetime_datetime', 'updated_at_datetime_datetime',
       'created_at_datetime_datetime', 'date_of_birth_datetime_datetime'],
      dtype='str')

In [47]:
df_users = df_users.drop(columns=['api_key','date_of_birth_datetime','info','oauth_sub','username','bio','gender','date_of_birth','info_parsed','date_of_birth_datetime_datetime']).head()

In [ ]:
df_users.head()

,id,name,email,role,profile_image_url,last_active_at,updated_at,created_at,settings,prolific_pid,...,current_session_id,session_number,consent_given,last_active_at_datetime,updated_at_datetime,created_at_datetime,settings_parsed,last_active_at_datetime_datetime,updated_at_datetime_datetime,created_at_datetime_datetime
0,055b5ebd-6a8b-4a75-b608-3c25f69a5197,haojian,prolific_haojian@prolific.study,user,/user.png,1768367761,1768345165,1768345125,{'ui': {'selectedChildId': '619ea48a-a376-4fce...,haojian,...,1,1,t,1970-01-21 11:12:47.761,1970-01-21 11:12:25.165,1970-01-21 11:12:25.125,{'ui': {'selectedChildId': '619ea48a-a376-4fce...,1970-01-21 11:12:47.761,1970-01-21 11:12:25.165,1970-01-21 11:12:25.125
1,46bf4ed4-ba1e-467c-bbcd-8ed67c68406c,Admin,childai.research.ucsd@gmail.com,admin,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",1768367096,1768329895,1768329895,null,NaN,...,NaN,1,f,1970-01-21 11:12:47.096,1970-01-21 11:12:09.895,1970-01-21 11:12:09.895,None,1970-01-21 11:12:47.096,1970-01-21 11:12:09.895,1970-01-21 11:12:09.895
2,1fc4965c-0c08-43b3-9bfd-02cbf5a02cf6,test1,prolific_test1@prolific.study,user,/user.png,1768368053,1768329978,1768329969,{'ui': {'selectedChildId': 'a4f0224b-2d70-4b92...,test1,...,1,1,t,1970-01-21 11:12:48.053,1970-01-21 11:12:09.978,1970-01-21 11:12:09.969,{'ui': {'selectedChildId': 'a4f0224b-2d70-4b92...,1970-01-21 11:12:48.053,1970-01-21 11:12:09.978,1970-01-21 11:12:09.969
3,16a92a7a-e4f1-46d2-91c4-c0e32208411c,lucas,prolific_lucas@prolific.study,user,/user.png,1768347875,1768345170,1768345157,{'ui': {'selectedChildId': '25d6122a-cdbc-40a8...,lucas,...,1,1,t,1970-01-21 11:12:27.875,1970-01-21 11:12:25.170,1970-01-21 11:12:25.157,{'ui': {'selectedChildId': '25d6122a-cdbc-40a8...,1970-01-21 11:12:27.875,1970-01-21 11:12:25.170,1970-01-21 11:12:25.157


### 5.3: Clean Chat Table

In [26]:
if 'chat' in raw_dataframes:
    print("Cleaning chat table...")
    df_chats = raw_dataframes['chat'].copy()
    
    # Basic cleaning
    df_chats = clean_strings(df_chats)
    df_chats = convert_timestamps(df_chats)
    
    # Parse JSON fields
    df_chats = parse_json_column(df_chats, 'chat')
    df_chats = parse_json_column(df_chats, 'meta')
    
    # Extract message count from chat JSON
    if 'chat_parsed' in df_chats.columns:
        def count_messages(chat_data):
            if isinstance(chat_data, dict):
                history = chat_data.get('history', {})
                messages = history.get('messages', {})
                if isinstance(messages, dict):
                    return len(messages)
            return 0
        
        df_chats['message_count'] = df_chats['chat_parsed'].apply(count_messages)
    
    # Convert boolean columns
    for col in ['archived', 'pinned']:
        if col in df_chats.columns:
            df_chats[col] = df_chats[col].astype(str).str.lower() == 'true'
    
    print(f"  ✓ Cleaned chat table: {len(df_chats)} rows, {len(df_chats.columns)} columns")
    print(f"  Total messages across all chats: {df_chats['message_count'].sum() if 'message_count' in df_chats.columns else 'N/A'}")
else:
    print("✗ Chat table not found")
    df_chats = pd.DataFrame()

✗ Chat table not found


### 5.4: Clean Message Table

In [27]:
if 'message' in raw_dataframes:
    print("Cleaning message table...")
    df_messages = raw_dataframes['message'].copy()
    
    # Basic cleaning
    df_messages = clean_strings(df_messages)
    df_messages = convert_timestamps(df_messages)
    
    # Parse JSON fields
    df_messages = parse_json_column(df_messages, 'data')
    df_messages = parse_json_column(df_messages, 'meta')
    
    print(f"  ✓ Cleaned message table: {len(df_messages)} rows, {len(df_messages.columns)} columns")
    if 'role' in df_messages.columns:
        print(f"  Messages by role:")
        print(df_messages['role'].value_counts().to_string())
else:
    print("✗ Message table not found")
    df_messages = pd.DataFrame()

✗ Message table not found


### 5.5: Clean Child Profile Table

In [14]:
if 'child_profile' in raw_dataframes:
    print("Cleaning child_profile table...")
    df_child_profiles = raw_dataframes['child_profile'].copy()
    
    # Basic cleaning
    df_child_profiles = clean_strings(df_child_profiles)
    df_child_profiles = convert_timestamps(df_child_profiles)
    
    # Parse JSON fields if any
    for col in df_child_profiles.columns:
        if df_child_profiles[col].dtype == 'object':
            sample = df_child_profiles[col].dropna().astype(str)
            if len(sample) > 0 and sample.str.startswith('{').any():
                df_child_profiles = parse_json_column(df_child_profiles, col)
    
    # Convert boolean columns
    for col in ['is_current', 'is_only_child']:
        if col in df_child_profiles.columns:
            df_child_profiles[col] = df_child_profiles[col].astype(str).str.lower() == 'true'
    
    # Convert numeric columns
    for col in ['attempt_number', 'session_number']:
        if col in df_child_profiles.columns:
            df_child_profiles[col] = pd.to_numeric(df_child_profiles[col], errors='coerce')
    
    print(f"  ✓ Cleaned child_profile table: {len(df_child_profiles)} rows, {len(df_child_profiles.columns)} columns")
    print(f"  Unique users: {df_child_profiles['user_id'].nunique() if 'user_id' in df_child_profiles.columns else 'N/A'}")
else:
    print("✗ Child profile table not found")
    df_child_profiles = pd.DataFrame()

Cleaning child_profile table...
  ✓ Cleaned child_profile table: 4 rows, 21 columns
  Unique users: 3


In [ ]:
df_child_profiles.head()

,id,user_id,name,child_age,child_gender,child_characteristics,created_at,updated_at,attempt_number,is_current,...,parent_llm_monitoring_level,child_gender_other,child_ai_use_contexts_other,parent_llm_monitoring_other,created_at_datetime,updated_at_datetime,attempt_number_datetime,created_at_datetime_datetime,updated_at_datetime_datetime,attempt_number_datetime_datetime
0,a4f0224b-2d70-4b92-bfa7-4caef02f5df4,1fc4965c-0c08-43b3-9bfd-02cbf5a02cf6,bob,10 years old,Non-binary,"Conscientiousness: Is systematic, likes to kee...",1768329996252018976,1768343088867177168,1,False,...,prefer_not_to_say,None,None,None,2026-01-13 18:46:36.252018929,2026-01-13 22:24:48.867177248,1970-01-01 00:00:01,2026-01-13 18:46:36.252018929,2026-01-13 22:24:48.867177248,1970-01-01 00:00:01
1,25d6122a-cdbc-40a8-817b-82edb4a75ef7,16a92a7a-e4f1-46d2-91c4-c0e32208411c,Izak,18 years old,Male,"Conscientiousness: Is systematic, likes to kee...",1768345250730407715,1768345250730407715,1,False,...,plan_to,None,None,None,2026-01-13 23:00:50.730407715,2026-01-13 23:00:50.730407715,1970-01-01 00:00:01,2026-01-13 23:00:50.730407715,2026-01-13 23:00:50.730407715,1970-01-01 00:00:01
2,de6a59d8-e18a-49da-b573-3277c1c50a0a,055b5ebd-6a8b-4a75-b608-3c25f69a5197,Izak,12 years old,Non-binary,"Agreeableness: Is compassionate, has a soft he...",1768345282627368961,1768345282627368961,1,False,...,plan_to,None,None,None,2026-01-13 23:01:22.627368927,2026-01-13 23:01:22.627368927,1970-01-01 00:00:01,2026-01-13 23:01:22.627368927,2026-01-13 23:01:22.627368927,1970-01-01 00:00:01
3,619ea48a-a376-4fce-96f7-9ff4c87c06fe,055b5ebd-6a8b-4a75-b608-3c25f69a5197,HAHAHA,10 years old,Male,"Conscientiousness: Is persistent, works until ...",1768345339787081033,1768345339787081033,1,False,...,occasional_guidance,None,None,None,2026-01-13 23:02:19.787081003,2026-01-13 23:02:19.787081003,1970-01-01 00:00:01,2026-01-13 23:02:19.787081003,2026-01-13 23:02:19.787081003,1970-01-01 00:00:01


### 5.6: Clean Selection Table

In [15]:
if 'selection' in raw_dataframes:
    print("Cleaning selection table...")
    df_selections = raw_dataframes['selection'].copy()
    
    # Basic cleaning
    df_selections = clean_strings(df_selections)
    df_selections = convert_timestamps(df_selections)
    
    # Parse JSON fields
    df_selections = parse_json_column(df_selections, 'meta')
    
    # Convert numeric columns
    for col in ['start_offset', 'end_offset']:
        if col in df_selections.columns:
            df_selections[col] = pd.to_numeric(df_selections[col], errors='coerce')
    
    print(f"  ✓ Cleaned selection table: {len(df_selections)} rows, {len(df_selections.columns)} columns")
    if 'role' in df_selections.columns:
        print(f"  Selections by role:")
        print(df_selections['role'].value_counts().to_string())
    if 'source' in df_selections.columns:
        print(f"  Selections by source:")
        print(df_selections['source'].value_counts().to_string())
else:
    print("✗ Selection table not found")
    df_selections = pd.DataFrame()

Cleaning selection table...
  ✓ Cleaned selection table: 47 rows, 20 columns
  Selections by role:
role
assistant    38
user          9
  Selections by source:
source
response    38
prompt       9


In [ ]:
df_selections = df_selections.drop(columns=['context','meta','start_offset','end_offset','chat_id_datetime','meta_parsed','chat_id_datetime_datetime'])

KeyError: "['context', 'meta', 'start_offset', 'end_offset', 'chat_id_datetime', 'meta_parsed', 'chat_id_datetime_datetime'] not found in axis"

In [ ]:
df_selections.head()

,id,user_id,chat_id,message_id,role,selected_text,created_at,updated_at,scenario_id,source,child_id,assignment_id,created_at_datetime,updated_at_datetime,created_at_datetime_datetime,updated_at_datetime_datetime
0,568a1b24-0f9f-4bd2-990c-a1f8b730e658,1fc4965c-0c08-43b3-9bfd-02cbf5a02cf6,assignment_dbeb40ea-dbf4-46d9-94df-9907660020b3,dbeb40ea-dbf4-46d9-94df-9907660020b3:prompt,user,"my dad posts pics of me and it's embarrassing,...",1768330003273891709,1768330003273891709,NaN,prompt,NaN,dbeb40ea-dbf4-46d9-94df-9907660020b3,2026-01-13 18:46:43.273891687,2026-01-13 18:46:43.273891687,2026-01-13 18:46:43.273891687,2026-01-13 18:46:43.273891687
1,e9990b1f-bbfc-40f8-908f-3794a5bc344f,1fc4965c-0c08-43b3-9bfd-02cbf5a02cf6,assignment_dbeb40ea-dbf4-46d9-94df-9907660020b3,dbeb40ea-dbf4-46d9-94df-9907660020b3:response,assistant,can (and should) tell him to stop or at least c,1768330007137792387,1768330007137792387,NaN,response,NaN,dbeb40ea-dbf4-46d9-94df-9907660020b3,2026-01-13 18:46:47.137792587,2026-01-13 18:46:47.137792587,2026-01-13 18:46:47.137792587,2026-01-13 18:46:47.137792587
2,adcd2bac-18e8-4f05-aa1e-b2d1847e30ac,1fc4965c-0c08-43b3-9bfd-02cbf5a02cf6,assignment_dbeb40ea-dbf4-46d9-94df-9907660020b3,dbeb40ea-dbf4-46d9-94df-9907660020b3:response,assistant,What to say (simple and respectful)\n\nTry usi...,1768330010682180329,1768330010682180329,NaN,response,NaN,dbeb40ea-dbf4-46d9-94df-9907660020b3,2026-01-13 18:46:50.682180405,2026-01-13 18:46:50.682180405,2026-01-13 18:46:50.682180405,2026-01-13 18:46:50.682180405
3,badfd993-4903-44c4-86c0-a41c745d9bcb,1fc4965c-0c08-43b3-9bfd-02cbf5a02cf6,assignment_dbeb40ea-dbf4-46d9-94df-9907660020b3,dbeb40ea-dbf4-46d9-94df-9907660020b3:response,assistant,Can you please ask me before you post?”,1768330015665036053,1768330015665036053,NaN,response,NaN,dbeb40ea-dbf4-46d9-94df-9907660020b3,2026-01-13 18:46:55.665035963,2026-01-13 18:46:55.665035963,2026-01-13 18:46:55.665035963,2026-01-13 18:46:55.665035963
4,5824fd70-717d-423f-88ec-12ecb4366be4,1fc4965c-0c08-43b3-9bfd-02cbf5a02cf6,assignment_dbeb40ea-dbf4-46d9-94df-9907660020b3,dbeb40ea-dbf4-46d9-94df-9907660020b3:response,assistant,Offer a compromise\n\nIf “stop completely” fee...,1768330019679754536,1768330019679754536,NaN,response,NaN,dbeb40ea-dbf4-46d9-94df-9907660020b3,2026-01-13 18:46:59.679754496,2026-01-13 18:46:59.679754496,2026-01-13 18:46:59.679754496,2026-01-13 18:46:59.679754496


### 5.7: Clean Moderation Tables

In [16]:
# Moderation Scenario
if 'moderation_scenario' in raw_dataframes:
    print("Cleaning moderation_scenario table...")
    df_mod_scenarios = raw_dataframes['moderation_scenario'].copy()
    df_mod_scenarios = clean_strings(df_mod_scenarios)
    df_mod_scenarios = convert_timestamps(df_mod_scenarios)
    
    # Convert boolean columns
    for col in ['is_applicable']:
        if col in df_mod_scenarios.columns:
            df_mod_scenarios[col] = df_mod_scenarios[col].astype(str).str.lower() == 'true'
    
    print(f"  ✓ Cleaned moderation_scenario: {len(df_mod_scenarios)} rows")
else:
    df_mod_scenarios = pd.DataFrame()

# Moderation Session
if 'moderation_session' in raw_dataframes:
    print("Cleaning moderation_session table...")
    df_mod_sessions = raw_dataframes['moderation_session'].copy()
    df_mod_sessions = clean_strings(df_mod_sessions)
    df_mod_sessions = convert_timestamps(df_mod_sessions)
    
    # Parse JSON fields
    json_cols = ['strategies', 'custom_instructions', 'highlighted_texts', 
                 'refactored_response', 'session_metadata']
    for col in json_cols:
        if col in df_mod_sessions.columns:
            df_mod_sessions = parse_json_column(df_mod_sessions, col)
    
    # Convert boolean and numeric columns
    if 'is_final_version' in df_mod_sessions.columns:
        df_mod_sessions['is_final_version'] = df_mod_sessions['is_final_version'].astype(str).str.lower() == 'true'
    for col in ['scenario_index', 'attempt_number', 'version_number']:
        if col in df_mod_sessions.columns:
            df_mod_sessions[col] = pd.to_numeric(df_mod_sessions[col], errors='coerce')
    
    print(f"  ✓ Cleaned moderation_session: {len(df_mod_sessions)} rows")
else:
    df_mod_sessions = pd.DataFrame()

# Moderation Applied
if 'moderation_applied' in raw_dataframes:
    print("Cleaning moderation_applied table...")
    df_mod_applied = raw_dataframes['moderation_applied'].copy()
    df_mod_applied = clean_strings(df_mod_applied)
    df_mod_applied = convert_timestamps(df_mod_applied)
    
    # Parse JSON fields
    json_cols = ['strategies', 'custom_instructions', 'highlighted_texts', 'refactored_response']
    for col in json_cols:
        if col in df_mod_applied.columns:
            df_mod_applied = parse_json_column(df_mod_applied, col)
    
    if 'confirmed_preferred' in df_mod_applied.columns:
        df_mod_applied['confirmed_preferred'] = df_mod_applied['confirmed_preferred'].astype(str).str.lower() == 'true'
    if 'version_index' in df_mod_applied.columns:
        df_mod_applied['version_index'] = pd.to_numeric(df_mod_applied['version_index'], errors='coerce')
    
    print(f"  ✓ Cleaned moderation_applied: {len(df_mod_applied)} rows")
else:
    df_mod_applied = pd.DataFrame()

# Moderation Question Answer
if 'moderation_question_answer' in raw_dataframes:
    print("Cleaning moderation_question_answer table...")
    df_mod_qa = raw_dataframes['moderation_question_answer'].copy()
    df_mod_qa = clean_strings(df_mod_qa)
    df_mod_qa = convert_timestamps(df_mod_qa)
    print(f"  ✓ Cleaned moderation_question_answer: {len(df_mod_qa)} rows")
else:
    df_mod_qa = pd.DataFrame()

Cleaning moderation_session table...
  ✓ Cleaned moderation_session: 32 rows


In [ ]:
df_mod_qa.head()

""


this should be nothing. it's fine. don't pass to output

### 5.8: Clean Exit Quiz Table

In [17]:
if 'exit_quiz_response' in raw_dataframes:
    print("Cleaning exit_quiz_response table...")
    df_exit_quiz = raw_dataframes['exit_quiz_response'].copy()
    
    # Basic cleaning
    df_exit_quiz = clean_strings(df_exit_quiz)
    df_exit_quiz = convert_timestamps(df_exit_quiz)
    
    # Parse JSON fields
    df_exit_quiz = parse_json_column(df_exit_quiz, 'answers')
    df_exit_quiz = parse_json_column(df_exit_quiz, 'score')
    df_exit_quiz = parse_json_column(df_exit_quiz, 'meta')
    
    # Convert boolean and numeric columns
    if 'is_current' in df_exit_quiz.columns:
        df_exit_quiz['is_current'] = df_exit_quiz['is_current'].astype(str).str.lower() == 'true'
    if 'attempt_number' in df_exit_quiz.columns:
        df_exit_quiz['attempt_number'] = pd.to_numeric(df_exit_quiz['attempt_number'], errors='coerce')
    
    print(f"  ✓ Cleaned exit_quiz_response table: {len(df_exit_quiz)} rows, {len(df_exit_quiz.columns)} columns")
else:
    print("✗ Exit quiz table not found")
    df_exit_quiz = pd.DataFrame()

Cleaning exit_quiz_response table...
  ✓ Cleaned exit_quiz_response table: 5 rows, 17 columns


In [ ]:
df_exit_quiz = df_exit_quiz.drop(columns=['answers','score','answers_parsed','score_parsed','meta_parsed'])


In [ ]:
df_exit_quiz.head()

,id,user_id,child_id,question_key,meta,created_at,updated_at,attempt_number,is_current,created_at_datetime,updated_at_datetime,attempt_number_datetime,created_at_datetime_datetime,updated_at_datetime_datetime,attempt_number_datetime_datetime
0,785ba1d3-1145-4cd2-8a10-a22673caf031,1fc4965c-0c08-43b3-9bfd-02cbf5a02cf6,a4f0224b-2d70-4b92-bfa7-4caef02f5df4,"{'parentGender': 'non-binary', 'parentAge': '4...",{'page': 'exit-survey'},1768343162551909785,1768343162551909785,1,False,2026-01-13 22:26:02.551909924,2026-01-13 22:26:02.551909924,1970-01-01 00:00:01,2026-01-13 22:26:02.551909924,2026-01-13 22:26:02.551909924,1970-01-01 00:00:01
1,90ed692c-c814-41f4-9c91-81d117a38809,055b5ebd-6a8b-4a75-b608-3c25f69a5197,de6a59d8-e18a-49da-b573-3277c1c50a0a,"{'parentGender': 'female', 'parentAge': '18-24...",{'page': 'exit-survey'},1768345610687093700,1768345610687093700,1,False,2026-01-13 23:06:50.687093735,2026-01-13 23:06:50.687093735,1970-01-01 00:00:01,2026-01-13 23:06:50.687093735,2026-01-13 23:06:50.687093735,1970-01-01 00:00:01
2,cb7e2e2f-83c9-4d57-a94b-9d108816e6ed,16a92a7a-e4f1-46d2-91c4-c0e32208411c,25d6122a-cdbc-40a8-817b-82edb4a75ef7,"{'parentGender': 'female', 'parentAge': '45-54...",{'page': 'exit-survey'},1768345647511205915,1768345647511205915,1,False,2026-01-13 23:07:27.511205912,2026-01-13 23:07:27.511205912,1970-01-01 00:00:01,2026-01-13 23:07:27.511205912,2026-01-13 23:07:27.511205912,1970-01-01 00:00:01
3,d437033d-b25b-4db6-9a8a-f74ce62b9f77,16a92a7a-e4f1-46d2-91c4-c0e32208411c,25d6122a-cdbc-40a8-817b-82edb4a75ef7,"{'parentGender': 'non-binary', 'parentAge': 'p...",{'page': 'exit-survey'},1768345697496201187,1768345697496201187,1,False,2026-01-13 23:08:17.496201277,2026-01-13 23:08:17.496201277,1970-01-01 00:00:01,2026-01-13 23:08:17.496201277,2026-01-13 23:08:17.496201277,1970-01-01 00:00:01
4,132c82f2-7c94-47bd-93b4-0ec2f56b2fca,055b5ebd-6a8b-4a75-b608-3c25f69a5197,619ea48a-a376-4fce-96f7-9ff4c87c06fe,"{'parentGender': 'male', 'parentAge': '35-44',...",{'page': 'exit-survey'},1768346110582149960,1768346110582149960,1,False,2026-01-13 23:15:10.582149982,2026-01-13 23:15:10.582149982,1970-01-01 00:00:01,2026-01-13 23:15:10.582149982,2026-01-13 23:15:10.582149982,1970-01-01 00:00:01


### 5.9: Clean Scenario Tables (if present)

In [ ]:
# Clean each scenario table independently

# 1. Scenario Assignments
if 'scenario_assignments' in raw_dataframes:
    print("="*60)
    print("Cleaning scenario_assignments table (INDEPENDENT)")
    print("="*60)
    df_scenario_assignments = raw_dataframes['scenario_assignments'].copy()
    print(f"  Raw: {len(df_scenario_assignments)} rows, {len(df_scenario_assignments.columns)} columns")
    
    df_scenario_assignments = clean_strings(df_scenario_assignments)
    df_scenario_assignments = convert_timestamps_enhanced(df_scenario_assignments)
    
    # Convert numeric columns
    numeric_cols = ['alpha', 'eligible_pool_size', 'n_assigned_before', 'weight', 
                   'sampling_prob', 'assignment_position', 'issue_any', 'duration_seconds']
    for col in numeric_cols:
        if col in df_scenario_assignments.columns:
            df_scenario_assignments[col] = pd.to_numeric(df_scenario_assignments[col], errors='coerce')
    
    # Parse JSON fields if any
    for col in df_scenario_assignments.columns:
        if df_scenario_assignments[col].dtype == 'object':
            sample = df_scenario_assignments[col].dropna().astype(str)
            if len(sample) > 0 and (sample.str.startswith('{').any() or sample.str.startswith('[').any()):
                df_scenario_assignments = parse_json_column_enhanced(df_scenario_assignments, col)
    
    print(f"  ✓ Cleaned scenario_assignments: {len(df_scenario_assignments)} rows")
    print()
else:
    df_scenario_assignments = pd.DataFrame()

# 2. Scenarios
if 'scenarios' in raw_dataframes:
    print("="*60)
    print("Cleaning scenarios table (INDEPENDENT)")
    print("="*60)
    df_scenarios = raw_dataframes['scenarios'].copy()
    print(f"  Raw: {len(df_scenarios)} rows, {len(df_scenarios.columns)} columns")
    
    df_scenarios = clean_strings(df_scenarios)
    df_scenarios = convert_timestamps_enhanced(df_scenarios)
    
    # Convert boolean and numeric columns
    if 'is_active' in df_scenarios.columns:
        df_scenarios['is_active'] = df_scenarios['is_active'].astype(str).str.lower() == 'true'
    numeric_cols = ['n_assigned', 'n_completed', 'n_skipped', 'n_abandoned']
    for col in numeric_cols:
        if col in df_scenarios.columns:
            df_scenarios[col] = pd.to_numeric(df_scenarios[col], errors='coerce')
    
    # Parse JSON fields if any
    for col in df_scenarios.columns:
        if df_scenarios[col].dtype == 'object':
            sample = df_scenarios[col].dropna().astype(str)
            if len(sample) > 0 and (sample.str.startswith('{').any() or sample.str.startswith('[').any()):
                df_scenarios = parse_json_column_enhanced(df_scenarios, col)
    
    print(f"  ✓ Cleaned scenarios: {len(df_scenarios)} rows")
    print()
else:
    df_scenarios = pd.DataFrame()

# 3. Attention Check Scenarios
if 'attention_check_scenarios' in raw_dataframes:
    print("="*60)
    print("Cleaning attention_check_scenarios table (INDEPENDENT)")
    print("="*60)
    df_attention_checks = raw_dataframes['attention_check_scenarios'].copy()
    print(f"  Raw: {len(df_attention_checks)} rows, {len(df_attention_checks.columns)} columns")
    
    df_attention_checks = clean_strings(df_attention_checks)
    df_attention_checks = convert_timestamps_enhanced(df_attention_checks)
    
    if 'is_active' in df_attention_checks.columns:
        df_attention_checks['is_active'] = df_attention_checks['is_active'].astype(str).str.lower() == 'true'
    
    # Parse JSON fields if any
    for col in df_attention_checks.columns:
        if df_attention_checks[col].dtype == 'object':
            sample = df_attention_checks[col].dropna().astype(str)
            if len(sample) > 0 and (sample.str.startswith('{').any() or sample.str.startswith('[').any()):
                df_attention_checks = parse_json_column_enhanced(df_attention_checks, col)
    
    print(f"  ✓ Cleaned attention_check_scenarios: {len(df_attention_checks)} rows")
    print()
else:
    df_attention_checks = pd.DataFrame()

# 4. Assignment Session Activity
if 'assignment_session_activity' in raw_dataframes:
    print("="*60)
    print("Cleaning assignment_session_activity table (INDEPENDENT)")
    print("="*60)
    df_activity = raw_dataframes['assignment_session_activity'].copy()
    print(f"  Raw: {len(df_activity)} rows, {len(df_activity.columns)} columns")
    
    df_activity = clean_strings(df_activity)
    df_activity = convert_timestamps_enhanced(df_activity)
    
    # Convert numeric columns
    numeric_cols = ['session_number', 'active_ms_delta', 'cumulative_ms']
    for col in numeric_cols:
        if col in df_activity.columns:
            df_activity[col] = pd.to_numeric(df_activity[col], errors='coerce')
    
    # Parse JSON fields if any
    for col in df_activity.columns:
        if df_activity[col].dtype == 'object':
            sample = df_activity[col].dropna().astype(str)
            if len(sample) > 0 and (sample.str.startswith('{').any() or sample.str.startswith('[').any()):
                df_activity = parse_json_column_enhanced(df_activity, col)
    
    print(f"  ✓ Cleaned assignment_session_activity: {len(df_activity)} rows")
    print()
else:
    df_activity = pd.DataFrame()

print("="*60)
print("All scenario tables cleaned independently")
print("="*60)

Cleaning scenario_assignments table (INDEPENDENT)
  Raw: 55 rows, 19 columns
  ✓ Cleaned scenario_assignments: 55 rows

Cleaning scenarios table (INDEPENDENT)
  Raw: 50 rows, 17 columns
  ✓ Cleaned scenarios: 50 rows

Cleaning attention_check_scenarios table (INDEPENDENT)
  Raw: 50 rows, 13 columns
  ✓ Cleaned attention_check_scenarios: 50 rows

Cleaning assignment_session_activity table (INDEPENDENT)
  Raw: 521 rows, 6 columns
  ✓ Cleaned assignment_session_activity: 521 rows

All scenario tables cleaned independently


In [61]:
df_scenario_assignments = df_scenario_assignments.drop(columns=['duration_seconds','status_datetime','duration_seconds_datetime'])

In [ ]:
df_scenario_assignments.head()

,assignment_id,participant_id,scenario_id,child_profile_id,status,assigned_at,started_at,ended_at,alpha,eligible_pool_size,...,weight,sampling_prob,assignment_position,issue_any,skip_stage,skip_reason,skip_reason_text,assigned_at_datetime,started_at_datetime,ended_at_datetime
0,5607a819-5380-48f4-8d58-cba79734a14b,055b5ebd-6a8b-4a75-b608-3c25f69a5197,scenario_5e747e63-ded9-4d3b-b51f-ab526af77781,619ea48a-a376-4fce-96f7-9ff4c87c06fe,skipped,1768364210294,1768364210761,1768364243983,1,32,...,0.5,0.022058823529411766,0,NaN,step1,not_applicable,User marked scenario as not applicable,1970-01-21 11:12:44.210294,1970-01-21 11:12:44.210761,1970-01-21 11:12:44.243983
1,3e9b6295-43f4-463e-af2a-815f434e836f,055b5ebd-6a8b-4a75-b608-3c25f69a5197,scenario_2d558a8a-700b-4dcc-98fe-ed5ba63cd9c6,619ea48a-a376-4fce-96f7-9ff4c87c06fe,started,1768364210654,1768364272460,NaN,1,27,...,0.5,0.026785714285714288,5,NaN,NaN,NaN,NaN,1970-01-21 11:12:44.210654,1970-01-21 11:12:44.272460,NaT
2,dbeb40ea-dbf4-46d9-94df-9907660020b3,1fc4965c-0c08-43b3-9bfd-02cbf5a02cf6,scenario_c68ebc57-bf31-4422-a3d1-89c1e988eddc,a4f0224b-2d70-4b92-bfa7-4caef02f5df4,completed,1768329996708,1768329999106,1768330057784,1,50,...,1,0.02,0,1,NaN,NaN,NaN,1970-01-21 11:12:09.996708,1970-01-21 11:12:09.999106,1970-01-21 11:12:10.057784
3,7b743ee2-c068-4f79-bc35-a2a60c473f4d,1fc4965c-0c08-43b3-9bfd-02cbf5a02cf6,scenario_72f1a8d3-c869-4951-912b-fffb534b6b88,a4f0224b-2d70-4b92-bfa7-4caef02f5df4,abandoned,1768329997346,1768330078974,1768331880280,1,47,...,1,0.02127659574468085,3,NaN,NaN,NaN,NaN,1970-01-21 11:12:09.997346,1970-01-21 11:12:10.078974,1970-01-21 11:12:11.880280
4,d00035e6-bede-48a0-bd46-df779facf441,1fc4965c-0c08-43b3-9bfd-02cbf5a02cf6,scenario_3b65b510-4be5-4f63-bfb0-647a2993bc14,a4f0224b-2d70-4b92-bfa7-4caef02f5df4,completed,1768329996915,1768333290370,1768333312549,1,49,...,1,0.02040816326530612,1,1,NaN,NaN,NaN,1970-01-21 11:12:09.996915,1970-01-21 11:12:13.290370,1970-01-21 11:12:13.312549


In [64]:
df_scenarios=df_scenarios.drop(columns=['trait','polarity','prompt_style','model_name'])

In [ ]:
df_scenarios.head()

,scenario_id,prompt_text,response_text,set_name,domain,source,is_active,n_assigned,n_completed,n_skipped,n_abandoned,created_at,updated_at,created_at_datetime,updated_at_datetime
0,scenario_4feb0c7f-37ca-41b0-9a2a-9f4282ea6df3,"what if lightning hits our house, will my bed ...",Lightning hitting a house can be scary to thin...,test,Casual Knowledge Domain,admin_upload,t,0,0,0,0,1768329928736,1768329928736,1970-01-21 11:12:09.928736,1970-01-21 11:12:09.928736
1,scenario_200798d9-c15c-45f0-81aa-300257f7876e,"I don't wanna hug grandma but she gets mad, sh...",You don’t have to hug anyone if you don’t want...,test,Casual Knowledge Domain,admin_upload,t,0,0,0,0,1768329928736,1768329928736,1970-01-21 11:12:09.928736,1970-01-21 11:12:09.928736
2,scenario_5f75dc66-dc34-4853-aa4f-5974cc1e5f3c,give me reasons I can tell my mom so she'll le...,"Here are some honest, respectful reasons you c...",test,Relationship Domain,admin_upload,t,0,0,0,0,1768329928736,1768329928736,1970-01-21 11:12:09.928736,1970-01-21 11:12:09.928736
3,scenario_bb8880ce-df1f-4a18-92fb-d86c2fe29026,how do I ask to play with them without them sa...,It really hurts when you ask and they say no. ...,test,Relationship Domain,admin_upload,t,1,0,0,0,1768329928736,1768364210522,1970-01-21 11:12:09.928736,1970-01-21 11:12:44.210522
4,scenario_c57a6e21-ec88-49f5-a51d-f0ba3d35730b,give me a science fair idea that I can actuall...,Here are a few science fair ideas you can fini...,test,Academic Domain,admin_upload,t,2,0,1,0,1768329928736,1768345917270,1970-01-21 11:12:09.928736,1970-01-21 11:12:25.917270


In [ ]:
df_attention_checks.drop(columns=['trait_theme','trait_phrase','sentiment','trait_index','prompt_index','sentiment_datetime'])

In [ ]:
df_attention_checks.head()

,scenario_id,prompt_text,response_text,set_name,is_active,source,created_at,updated_at,created_at_datetime,updated_at_datetime
0,ac_aa954a2c-6955-40e7-aea2-941fb665167a,what if you see a ghost but you think its your...,Sometimes our brains can “trick” us—especially...,test,t,admin_upload,1768329941909,1768329941909,1970-01-21 11:12:09.941909,1970-01-21 11:12:09.941909
1,ac_c645a8c5-50c5-4a1b-b0b6-b1dc326a3083,How do I stop procrastinating my college essay...,That feeling is super common. Your brain isn’t...,test,t,admin_upload,1768329941909,1768329941909,1970-01-21 11:12:09.941909,1970-01-21 11:12:09.941909
2,ac_af07da7b-97d8-4613-9520-4bfea726db56,"Why does my bike wobble more when I go slow, i...","When you ride a bike, it’s normal for it to fe...",test,t,admin_upload,1768329941909,1768329941909,1970-01-21 11:12:09.941909,1970-01-21 11:12:09.941909
3,ac_303cdc8f-b3a9-4391-ac48-a472e3602e59,How do I correct my teacher about my name with...,You can correct your teacher in a way that’s q...,test,t,admin_upload,1768329941909,1768329941909,1970-01-21 11:12:09.941909,1970-01-21 11:12:09.941909
4,ac_fad5329d-1395-456e-bdd0-ec477b431e40,Nobody lets me in their game and it makes my t...,That sounds really hard. When people don’t let...,test,t,admin_upload,1768329941909,1768329941909,1970-01-21 11:12:09.941909,1970-01-21 11:12:09.941909


In [ ]:
df_activity.head()

,id,user_id,session_number,active_ms_delta,cumulative_ms,created_at,cumulative_ms_datetime,created_at_datetime
0,50221205-492d-4d84-8e2a-7237210fcce2,1fc4965c-0c08-43b3-9bfd-02cbf5a02cf6,1,0,0,1768329980845,1970-01-01 00:00:00,1970-01-21 11:12:09.980845
1,855be001-5646-4024-818c-974251c60746,1fc4965c-0c08-43b3-9bfd-02cbf5a02cf6,1,1000,1000,1768329981950,1970-01-01 00:16:40,1970-01-21 11:12:09.981950
2,f33b92c8-aaf6-45a4-86e5-f5543a057b9a,1fc4965c-0c08-43b3-9bfd-02cbf5a02cf6,1,0,1000,1768329981975,1970-01-01 00:16:40,1970-01-21 11:12:09.981975
3,14de855f-161f-46ee-afb0-7b6db3586cd1,1fc4965c-0c08-43b3-9bfd-02cbf5a02cf6,1,16000,17000,1768329998047,1970-01-01 04:43:20,1970-01-21 11:12:09.998047
4,6975c74e-e3e7-48a5-9e17-71b3ac8885e7,1fc4965c-0c08-43b3-9bfd-02cbf5a02cf6,1,0,17000,1768329998082,1970-01-01 04:43:20,1970-01-21 11:12:09.998082


## Step 7: Save Cleaned Data

In [74]:
cleaned_tables = [
    (df_users, 'df_users'),
    (df_child_profiles, 'df_child_profiles'),
    (df_selections, 'df_selections'),
    (df_exit_quiz, 'df_exit_quiz'),
    (df_scenario_assignments, 'df_scenario_assignments'),
    (df_scenarios, 'df_scenarios'),
    (df_attention_checks, 'df_attention_checks'),
    (df_activity, 'df_activity')
]

## Step 6.5: Explore Scenario Tables

Display the head of all scenario-related tables to understand the data structure.

In [ ]:
# Display head of all scenario-related tables
print("="*70)
print("SCENARIO TABLES OVERVIEW")
print("="*70)

scenario_tables = {
    'scenarios': df_scenarios if 'df_scenarios' in globals() else None,
    'scenario_assignments': df_scenario_assignments if 'df_scenario_assignments' in globals() else None,
    'attention_check_scenarios': df_attention_checks if 'df_attention_checks' in globals() else None,
    'assignment_session_activity': df_session_activity if 'df_session_activity' in globals() else None,
}

for table_name, df in scenario_tables.items():
    print(f"\n{'='*70}")
    print(f"TABLE: {table_name}")
    print(f"{'='*70}")
    
    if df is not None and len(df) > 0:
        print(f"Shape: {df.shape[0]} rows × {df.shape[1]} columns")
        print(f"\nColumns: {', '.join(df.columns.tolist())}")
        print(f"\nFirst 3 rows:")
        print("-"*70)
        display(df.head(3))
    else:
        print("⚠️  Table not found or empty")

print(f"\n{'='*70}")
print("SCENARIO TABLES SUMMARY")
print(f"{'='*70}")
for table_name, df in scenario_tables.items():
    if df is not None and len(df) > 0:
        print(f"✓ {table_name}: {len(df)} rows")
    else:
        print(f"⚪ {table_name}: empty or not loaded")

In [79]:
# Create output directory with timestamp
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = Path("data-exports") / timestamp
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Saving cleaned DataFrames to {output_dir}/...")

saved_files = {}
for df, table_name in cleaned_tables:
    if len(df) > 0:
        # Save as pickle only (for git upload)
        pkl_file = output_dir / f"{table_name}.pkl"
        df.to_pickle(pkl_file)
        
        saved_files[table_name] = {
            'pkl': str(pkl_file),
            'rows': len(df),
            'columns': len(df.columns)
        }
        print(f"  ✓ {table_name}: {len(df)} rows × {len(df.columns)} cols -> {pkl_file.name}")

print(f"\n✓ Saved {len(saved_files)} tables to {output_dir}")

# Create summary JSON
summary = {
    'extraction_date': datetime.now().isoformat(),
    'dump_file': str(dump_file) if 'dump_file' in locals() else None,
    'tables': {}
}

for df, table_name in cleaned_tables:
    if len(df) > 0:
        summary['tables'][table_name] = {
            'row_count': len(df),
            'column_count': len(df.columns),
            'columns': list(df.columns),
            'memory_usage_mb': round(df.memory_usage(deep=True).sum() / 1024 / 1024, 2)
        }

summary_file = output_dir / "summary.json"
with open(summary_file, 'w') as f:
    json.dump(summary, f, indent=2)

print(f"✓ Summary saved to {summary_file.name}")
print(f"\nOutput location: {output_dir}")

Saving cleaned DataFrames to data_exports/...
  ✓ df_users: 4 rows -> df_users.csv, df_users.pkl
  ✓ df_child_profiles: 4 rows -> df_child_profiles.csv, df_child_profiles.pkl
  ✓ df_selections: 47 rows -> df_selections.csv, df_selections.pkl
  ✓ df_exit_quiz: 5 rows -> df_exit_quiz.csv, df_exit_quiz.pkl
  ✓ df_scenario_assignments: 55 rows -> df_scenario_assignments.csv, df_scenario_assignments.pkl
  ✓ df_scenarios: 50 rows -> df_scenarios.csv, df_scenarios.pkl
  ✓ df_attention_checks: 50 rows -> df_attention_checks.csv, df_attention_checks.pkl
  ✓ df_activity: 521 rows -> df_activity.csv, df_activity.pkl

✓ Saved 8 tables
✓ Summary saved to data_exports/summary.json


## Conclusion

All relevant tables have been extracted, cleaned, and saved to the `data_exports/` directory.

You can now load the cleaned data using:
```python
import pandas as pd
df = pd.read_pickle('data_exports/table_name.pkl')
```

The data has been:
- Parsed from the PostgreSQL dump
- Cleaned (null bytes removed, strings normalized)
- Transformed (timestamps converted, JSON parsed, types corrected)
- Saved in both CSV and pickle formats for easy access